In [1]:
print("hello")


hello


In [48]:
import sys
!{sys.executable} -m pip install -U numpy
!{sys.executable} -m pip install -U gensim
!{sys.executable} -m pip install -U python-Levenshtein
!{sys.executable} -m pip install -U nltk
!{sys.executable} -m pip install -U scikit-learn
!{sys.executable} -m pip install -U pandas
!{sys.executable} -m pip install -U gensim
#Import all the dependencies
import warnings
#warnings.filterwarnings("ignore")
import nltk
nltk.download('punkt')

from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
from numpy import *
from urllib.parse import unquote

import numpy as np
import pandas as pd
import csv
import urllib.parse as parse
import pickle


[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


  Obtaining dependency information for pandas from https://files.pythonhosted.org/packages/ae/d9/3741b344f57484b423cd22194025a8489992ad9962196a62721ef9980045/pandas-2.1.4-cp312-cp312-win_amd64.whl.metadata
  Obtaining dependency information for pytz>=2020.1 from https://files.pythonhosted.org/packages/32/4d/aaf7eff5deb402fd9a24a1449a8119f00d74ae9c2efa79f8ef9994261fc2/pytz-2023.3.post1-py2.py3-none-any.whl.metadata
     ---------------------------------------- 0.0/341.8 kB ? eta -:--:--
     --- --------------------------------- 30.7/341.8 kB 660.6 kB/s eta 0:00:01
     --- --------------------------------- 30.7/341.8 kB 660.6 kB/s eta 0:00:01
     ------ ------------------------------ 61.4/341.8 kB 469.7 kB/s eta 0:00:01
     ----------- ------------------------ 112.6/341.8 kB 656.4 kB/s eta 0:00:01
     --------------- -------------------- 143.4/341.8 kB 655.8 kB/s eta 0:00:01
     ------------------ ----------------- 174.1/341.8 kB 700.2 kB/s eta 0:00:01
     --------------------- --


[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!

[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [24]:
testXSS = []
testNORM = []
X_temp = []
X = []
y = []
xssnum = 0
notxssnum = 0
import urllib.parse  # Import the correct module for Python 3

print("Gathering Data...")
# gather the XSS string and append the label of 1 to y array
with open('testXSS.txt', 'r') as f:
    testXSS = f.readlines()
print("*", sep=' ', end='', flush=True)
# parse out the query part of the URL 
for line in testXSS:
   # query = parse.urlsplit(line)[3]
    query = urllib.parse.urlsplit(line)[3] 
    #try to remove open redirect vulns
    if "?http" in str(line):
        continue
    if "?url=http" in str(line):
        continue
    if "?fwd=http" in str(line):
        continue
    if "?path=http" in str(line):
        continue
    if "=http" in str(query):
        continue
    if "page=search" in str(query):
        continue
    if len(query) > 8:
        xssnum += 1
        #X_temp.append(query)
        X_temp.append(line)
        
# remove duplicates
dedup = list(dict.fromkeys(X_temp))
print("*", sep=' ', end='', flush=True)
# Add a feature to X and label to the y array
for line in dedup:
    #print("XSS => "+line)
    X.append(line)
    y.append(1)
    
X_temp = []
dedup = []
print("*", sep=' ', end='', flush=True)

# gather the list of normal string and append the label of 0 to y array 
with open('testNORM.txt', 'r', encoding='utf-8') as f:
    testNORM = f.readlines()
    
# parse out the query part of the URL 
for line in testNORM:
    query = urllib.parse.urlsplit(line)[3]  # Use urllib.parse.urlsplit
    #if "http" in str(query):
    #    continue
    if len(query) > 3:
        notxssnum += 1
        X_temp.append(line)
        
# remove duplicates
dedup = list(dict.fromkeys(X_temp))
print("*", sep=' ', end='', flush=True)
# Add a feature to X and a label to the y array
for line in dedup:
    #print("NOT XSS => "+line)
    X.append(line)
    y.append(0)

print("Number of XSS Samples: "+str(xssnum))
print("Number of NOT XSS Samples: "+str(notxssnum))
print("Total Samples: "+str(xssnum+notxssnum))



Gathering Data...
****Number of XSS Samples: 38608
Number of NOT XSS Samples: 44826
Total Samples: 83434


In [50]:
# Create a function to convert an array of query strings to a set of features
print("i am here")

from gensim.models.doc2vec import TaggedDocument
from nltk.tokenize import word_tokenize
# Create a function to convert an array of query strings to a set of features
def getVec(text):
    tagged_data = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(text)]
    max_epochs = 25
    vec_size = 20
    alpha = 0.025

    model = Doc2Vec(vector_size=vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                dm=1)
    
    model.build_vocab(tagged_data)
    print("Building the sample vector model...")
    features = []
    for epoch in range(max_epochs):
        #print('Doc2Vec Iteration {0}'.format(epoch))
        print("*", sep=' ', end='', flush=True)
        model.random.seed(42)
        model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=model.epochs)
        # decrease the learning rate
        model.alpha -= 0.0002
        # fix the learning rate, no decay
        model.min_alpha = model.alpha
    model.save("d2v.model")
    print()
    print("Model Saved")
    for i, line in enumerate(text):
        featureVec = [model.dv[i]]
        lineDecode = unquote(line)
        lineDecode = lineDecode.replace(" ", "")
        lowerStr = str(lineDecode).lower()
        #print("X"+str(i)+"=> "+line)
        # We could expand the features
        # https://websitesetup.org/javascript-cheat-sheet/
        # https://owasp.org/www-community/xss-filter-evasion-cheatsheet
        # https://html5sec.org/
        
        # add feature for malicious HTML tag count
        feature1 = int(lowerStr.count('<link'))
        feature1 += int(lowerStr.count('<object'))
        feature1 += int(lowerStr.count('<form'))
        feature1 += int(lowerStr.count('<embed'))
        feature1 += int(lowerStr.count('<ilayer'))
        feature1 += int(lowerStr.count('<layer'))
        feature1 += int(lowerStr.count('<style'))
        feature1 += int(lowerStr.count('<applet'))
        feature1 += int(lowerStr.count('<meta'))
        feature1 += int(lowerStr.count('<img'))
        feature1 += int(lowerStr.count('<iframe'))
        feature1 += int(lowerStr.count('<input'))
        feature1 += int(lowerStr.count('<body'))
        feature1 += int(lowerStr.count('<video'))
        feature1 += int(lowerStr.count('<button'))
        feature1 += int(lowerStr.count('<math'))
        feature1 += int(lowerStr.count('<picture'))
        feature1 += int(lowerStr.count('<map'))
        feature1 += int(lowerStr.count('<svg'))
        feature1 += int(lowerStr.count('<div'))
        feature1 += int(lowerStr.count('<a'))
        feature1 += int(lowerStr.count('<details'))
        feature1 += int(lowerStr.count('<frameset'))
        feature1 += int(lowerStr.count('<table'))
        feature1 += int(lowerStr.count('<comment'))
        feature1 += int(lowerStr.count('<base'))
        feature1 += int(lowerStr.count('<image'))
        # add feature for malicious method/event count
        feature2 = int(lowerStr.count('exec'))
        feature2 += int(lowerStr.count('fromcharcode'))
        feature2 += int(lowerStr.count('eval'))
        feature2 += int(lowerStr.count('alert'))
        feature2 += int(lowerStr.count('getelementsbytagname'))
        feature2 += int(lowerStr.count('write'))
        feature2 += int(lowerStr.count('unescape'))
        feature2 += int(lowerStr.count('escape'))
        feature2 += int(lowerStr.count('prompt'))
        feature2 += int(lowerStr.count('onload'))
        feature2 += int(lowerStr.count('onclick'))
        feature2 += int(lowerStr.count('onerror'))
        feature2 += int(lowerStr.count('onpage'))
        feature2 += int(lowerStr.count('confirm'))
        feature2 += int(lowerStr.count('marquee'))
        # add feature for ".js" count
        feature3 = int(lowerStr.count('.js'))
        # add feature for "javascript" count
        feature4 = int(lowerStr.count('javascript'))
        # add feature for length of the string
        feature5 = int(len(lowerStr))
        # add feature for "<script"  count
        feature6 = int(lowerStr.count('<script'))
        feature6 += int(lowerStr.count('&lt;script'))
        feature6 += int(lowerStr.count('%3cscript'))
        feature6 += int(lowerStr.count('%3c%73%63%72%69%70%74'))
        # add feature for special character count
        feature7 = int(lowerStr.count('&'))
        feature7 += int(lowerStr.count('<'))
        feature7 += int(lowerStr.count('>'))
        feature7 += int(lowerStr.count('"'))
        feature7 += int(lowerStr.count('\''))
        feature7 += int(lowerStr.count('/'))
        feature7 += int(lowerStr.count('%'))
        feature7 += int(lowerStr.count('*'))
        feature7 += int(lowerStr.count(';'))
        feature7 += int(lowerStr.count('+'))
        feature7 += int(lowerStr.count('='))
        feature7 += int(lowerStr.count('%3C'))
        # add feature for http count
        feature8 = int(lowerStr.count('http'))
        
        # append the features
        featureVec = np.append(featureVec,feature1)
        #featureVec = np.append(featureVec,feature2)
        featureVec = np.append(featureVec,feature3)
        featureVec = np.append(featureVec,feature4)
        featureVec = np.append(featureVec,feature5)
        featureVec = np.append(featureVec,feature6)
        featureVec = np.append(featureVec,feature7)
        #featureVec = np.append(featureVec,feature8)
        #print(featureVec)
        features.append(featureVec)
    return features



i am here


In [51]:
testText = [
                'Search=%22%3E\'%3E%3CSCRIPT%20SRC=http://br.zone-h.org/testes/xss.js%3E%3C/SCRIPT%3E?',
                'id=15%3Cscript%3Ealert%28document.cookie%29%3C/script%3E',
                'composite_search=1&keyword="/><script>alert("Xss:Vijayendra")</script>'
            ]
Xnew = getVec(testText)



Building the sample vector model...
*************************
Model Saved


In [55]:
print("Test Sample: "+ X[0])
print("Features: " + str(Xnew[0]))
print("\nLabel:\033[1;31;1m XSS(1)/\033[1;32;1m NOT XSS(0)\033[0;0m: " + str(y[0]))

Test Sample: http://search.rin.ru/cgi-bin/find.cgi?text=%3Cscript%3Ealert(%27HZ+iz+1337%27)%3B%3C%2Fscript%3E

Features: [-6.24447763e-01 -4.13311034e-01 -6.01661682e-01  5.03213704e-01
  6.36102855e-01 -5.52833617e-01 -4.35218006e-01  6.05183132e-02
 -1.00900781e+00 -4.31261390e-01  8.01654100e-01  3.58711034e-01
 -7.71161079e-01 -7.21224248e-01 -5.78382909e-01 -5.30140162e-01
  5.66670835e-01  3.06191474e-01 -1.28623760e+00 -6.65638626e-01
  0.00000000e+00  1.00000000e+00  0.00000000e+00  6.70000000e+01
  1.00000000e+00  1.50000000e+01]

Label: XSS(1)/ NOT XSS(0): 0


In [56]:
np.random.seed(42)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(Xnew, y, test_size = .3, random_state=42)

In [62]:
# Use RandomState for reproducibility.
from sklearn import tree
my_classifier1 = tree.DecisionTreeClassifier(random_state=42)
print(my_classifier1)
print()

from sklearn.svm import SVC
my_classifier2 = SVC(kernel='linear', random_state=42)
print(my_classifier2)
print()

from sklearn.naive_bayes import GaussianNB
my_classifier3 = GaussianNB()
print(my_classifier3)
print()

from sklearn.neighbors import KNeighborsClassifier
my_classifier4 = KNeighborsClassifier(n_neighbors=25, weights='uniform')
print(my_classifier4)
print()

from sklearn.ensemble import RandomForestClassifier
my_classifier5 = RandomForestClassifier(random_state=42)
print(my_classifier5)
print()

from sklearn.neural_network import MLPClassifier
my_classifier6 = MLPClassifier(max_iter=2000, random_state=42)
print(my_classifier6)
print()





DecisionTreeClassifier(random_state=42)

SVC(kernel='linear', random_state=42)

GaussianNB()

KNeighborsClassifier(n_neighbors=25)

RandomForestClassifier(random_state=42)

MLPClassifier(max_iter=2000, random_state=42)



In [69]:
print("Training Classifier #1 DecisionTreeClassifier")
my_classifier1.fit(X_train, y_train)
predictions1 = my_classifier1.predict(X_test)

Training Classifier #1 DecisionTreeClassifier


In [70]:
from sklearn.metrics import accuracy_score
print('Accuracy Score #1: {:.1%}'.format(accuracy_score(y_test, predictions1)))


Accuracy Score #1: 100.0%


In [71]:
from sklearn.metrics import classification_report
print("Classification Report #1 DecisionTreeClassifier")
print(classification_report(y_test, predictions1))


Classification Report #1 DecisionTreeClassifier
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         1

    accuracy                           1.00         1
   macro avg       1.00      1.00      1.00         1
weighted avg       1.00      1.00      1.00         1



In [73]:
print("Training Classifier #5 RandomForestClassifier")
my_classifier5.fit(X_train, y_train)


Training Classifier #5 RandomForestClassifier


RandomForestClassifier(random_state=42)

In [74]:
predictions5 = my_classifier5.predict(X_test)

In [75]:
print('Accuracy Score #5: {:.1%}'.format(accuracy_score(y_test, predictions5)))

Accuracy Score #5: 100.0%


In [76]:
print("Classification Report #5 RandomForestClassifier")
print(classification_report(y_test, predictions5))


Classification Report #5 RandomForestClassifier
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         1

    accuracy                           1.00         1
   macro avg       1.00      1.00      1.00         1
weighted avg       1.00      1.00      1.00         1



In [77]:
print("Training Classifier #4 KNeighborsClassifier")
my_classifier4.fit(X_train, y_train)


Training Classifier #4 KNeighborsClassifier


KNeighborsClassifier(n_neighbors=25)

In [80]:
from sklearn.metrics import confusion_matrix
print("\nConfusion Matrix #5 RandomForestClassifier")
print(confusion_matrix(y_test, predictions5))



Confusion Matrix #5 RandomForestClassifier
[[1]]


In [82]:
print("Training Classifier #5 RandomForestClassifier")
my_classifier5.fit(Xnew, y)


Training Classifier #5 RandomForestClassifier


RandomForestClassifier(random_state=42)

In [84]:
filename5 = 'RandomForestClassifier.sav'
pickle.dump(my_classifier5, open(filename5, 'wb'))


In [85]:
loaded_model5 = pickle.load(open(filename5, 'rb'))

In [86]:
testXSS = [
                '<script>alert(\'xss\')</script><script><script>',
                'hellomo',
                'https://store.bentley.com/en/shop/search?term=%22%3E%3Cdetails%20open%20ontoggle=prompt(1337)%3ExxLouisLouisLouis',
                'ghfdhgdhjgd',
                'uid%3D19%26list_page%3D%22%3E%3Cscript%3Ealert%28document.cookie%29%3B%3C/script%3E',
                '&template=en_search_error&postalCode=\\\';alert(0)//',
                '&where=%3Cscript%3Ealert%28%27xss%27%29%3C%2Fscript%3E&loctypes=1003%2C1001%2C1000%2C1%2C9%2C5%2C11%2C13%2C19%2C20&from=hdr_localsearch',
                'http://mydata.com/sad/sd/qwd/qwde/qwe/?sessionid=12',
                'http://mydata.com?id=script',
                '&\';}},{scope:\'email,user_about_me,user_hometown,user_interests,user_likes,user_status,user_website,user_birthday,publish_stream,publish_actions,offline_access\'});}alert(0);b=function(response){c=({a:{//',
                'http://myurl.com?<script',
                'http://mydata.com?script=script',
                'composite_search=1&keyword="/><script>alert("Xss:Vijayendra")</script>',
                'http://mysite.com?srtalert',
                'script',
                'alert',
                'Search=%22%3E\'%3E%3CSCRIPT%20SRC=http://br.zone-h.org/testes/xss.js%3E%3C/SCRIPT%3E?',
                'id=15%3Cscript%3Ealert%28document.cookie%29%3C/script%3E',
                'composite_search=1&keyword="/><script>alert("Xss:Vijayendra")</script>',
                'id=123&href=abdc<a<script>alert(1)',
                '<<<<<<>>>>></>,><><>',
                'alert()alert()',
                'alertalert',
                '?url=http://localhost:8888/notebooks/Documents/MachineLearning/Practical%20Machine%20Learning',
                '<script<script',
                '<scriptalert',
                'httphttphttp',
                'https://disqus.com/?ref_noscript',
                'I am a string',
                '<img src="javascript:alert(1)/>"',
                'HelloWorld!',
                'http://mysite.com?<script>',
                '<input type="text" value=`` <div/onmouseover=\'alert(471)\'>X</div>',
                '<img \x47src=x onerror="javascript:alert(324)">',
                '<a href="\xE2\x80\x87javascript:javascript:alert(183)" id="fuzzelement1">test</a>',
                '<body onscroll=javascript:alert(288)><br><br><br><br><br><br>...<br><br><br><br><br><br><br><br><br><br>...<br><br><br><br><br><br><br><br><br><br>...<br><br><br><br><br><br><br><br><br><br>...<br><br><br><br><br><br><br><br><br><br>...<br><br><br><br><input autofocus>',
                '<meta charset="mac-farsi">¼script¾javascript:alert(379)¼/script¾',
                '<HTML xmlns:xss><?import namespace=(493)s" implementation="%(htc)s"><xss:xss>XSS</xss:xss></HTML>""","XML namespace."),("""<XML ID=(494)s"><I><B>&lt;IMG SRC="javas<!-- -->cript:javascript:alert(420)"&gt;</B></I></XML><SPAN DATASRC="#xss" DATAFLD="B" DATAFORMATAS="HTML"></SPAN>'
            ]


In [87]:
Xnew = getVec(testXSS)
# make a prediction

ynew5 = loaded_model5.predict(Xnew)


Building the sample vector model...
*************************
Model Saved


In [89]:
xssCount = 0 
notXssCount = 0
for i in range(len(Xnew)):
    score = (.25*ynew5[i])
    print(ynew5[i])
    
    print(score)
    if score >= .5:
        print("\033[1;31;1mXSS\033[0;0m => "+testXSS[i])
        xssCount += 1
    else:
        print("\033[1;32;1mNOT XSS\033[0;0m => "+testXSS[i])
        notXssCount += 1

print()
print("*------------- RESULTS -------------*")
print("\033[1;31;1mXSS\033[0;0m => "+str(xssCount))
print("\033[1;32;1mNOT XSS\033[0;0m => "+str(notXssCount))

0
0.0
NOT XSS => <script>alert('xss')</script><script><script>
0
0.0
NOT XSS => hellomo
0
0.0
NOT XSS => https://store.bentley.com/en/shop/search?term=%22%3E%3Cdetails%20open%20ontoggle=prompt(1337)%3ExxLouisLouisLouis
0
0.0
NOT XSS => ghfdhgdhjgd
0
0.0
NOT XSS => uid%3D19%26list_page%3D%22%3E%3Cscript%3Ealert%28document.cookie%29%3B%3C/script%3E
0
0.0
NOT XSS => &template=en_search_error&postalCode=\';alert(0)//
0
0.0
NOT XSS => &where=%3Cscript%3Ealert%28%27xss%27%29%3C%2Fscript%3E&loctypes=1003%2C1001%2C1000%2C1%2C9%2C5%2C11%2C13%2C19%2C20&from=hdr_localsearch
0
0.0
NOT XSS => http://mydata.com/sad/sd/qwd/qwde/qwe/?sessionid=12
0
0.0
NOT XSS => http://mydata.com?id=script
0
0.0
NOT XSS => &';}},{scope:'email,user_about_me,user_hometown,user_interests,user_likes,user_status,user_website,user_birthday,publish_stream,publish_actions,offline_access'});}alert(0);b=function(response){c=({a:{//
0
0.0
NOT XSS => http://myurl.com?<script
0
0.0
NOT XSS => http://mydata.com?script=script
0
0.0